In [1]:
import pandas as pd
import numpy as np

ori_csv_path="appstore_games.csv"
ori_csv=pd.read_csv(ori_csv_path)

# Delete irrelevant columns
del ori_csv['URL']
del ori_csv['Subtitle']
del ori_csv['Icon URL']
del ori_csv['Description']
del ori_csv['Developer']
del ori_csv['Age Rating']
del ori_csv['Languages']
del ori_csv['Original Release Date']
del ori_csv['Current Version Release Date']

# Delete empty reviews
ori_csv=ori_csv.dropna(subset=['User Rating Count','Average User Rating'])

# Add a filter for rating count
ori_csv['filter']=1000
ori_csv.drop(ori_csv[ori_csv['User Rating Count']<ori_csv['filter']].index,inplace=True)
del ori_csv['filter']

# Delete genres other than Game
ori_csv.drop(ori_csv[ori_csv['Primary Genre']!='Games'].index,inplace=True)
del ori_csv['Primary Genre']

# Reset index
ori_csv=ori_csv.reset_index()
del ori_csv['index']

# Create a data frame for sub genres
subgenres=ori_csv['Genres'].str.split(', ',expand=True)
del subgenres[0]

# Form a list of unique sub genres
subgenre_list=[]
for i,row in subgenres.iterrows():
    for value in row:
        if value not in subgenre_list:
            if value != None:
                subgenre_list.append(value)

# Delete 'Strategy' since every game in the dataset belongs to the genre                
subgenre_list.remove('Strategy')

# Create a dummy variable data frame for the sub genres
zeros=np.zeros(shape=(len(subgenres),len(subgenre_list)))
subgenre_df=pd.DataFrame(zeros,columns=subgenre_list,index=ori_csv['ID'])

# Loop through the sub genres to create the dummy variables 
for k in range(len(subgenre_list)):
    for i in range(len(subgenres)):
        for j in range(len(subgenres.columns)):

            if subgenre_list[k] == subgenres.iloc[i,j]:
                subgenre_df.iloc[i,k]=1
            else:
                if subgenre_df.iloc[i,k]==1:
                    pass
                else:
                    subgenre_df.iloc[i,k]=0
subgenre_dummies=subgenre_df.iloc[:,:-2]

# Merge into a whole data frame
data=ori_csv.merge(subgenre_dummies,on=['ID'])
del data['Genres']

data.to_csv("data.csv")